# Part 1:
## Getting a GeoJSON file for each county in WA, containing all of its census blocks

This first part took FOREVER AND A HALF. I was working with Andrew Lonie for about 4 or 5 hours on the night of 01-28, and then it took another 3 or 4 hours with much help from you guys (Bergman and Lizotte) as I'm sure you're aware. >_< 

The hardest parts to figure out were how to add a matching census block row to the new .shp file. At first I tried InsertCursor inserting blank lines then going back with UpdateCursor to update every blank row with the matching row in "saep_bg10.shp" because from reading the book it made it sound like InsertCursor only made blank rows.

Then we tried using an append function, but that only appends whole .shp files not singled out rows...

But it turned out InsertCursor does in fact import rows, so that ended up working well.

Process:

1. Basically I started with a loop to gather up all the unique FIPS values for Washington State. 
2. Then, I used that loop as a counter for creating the appropriate amount of shape files (later to be .geojson). 
3. For each run of the master loop:
    1. a loop on the WashingtonFIPS.dbf is made to determine the proper county name associated with the FIPS code. 
    2. After the county name is found a polygon .shp mimicking the schema and spatial ref of the "saep_bg10.shp" is made with said county name. 
    3. Then one last loop through the "saep_bg10.shp" file is made to find each row that matches with the current working FIPS code so that it can be added to the county_name.shp file
    4. After all the matching rows from the "saep_bg10.shp" file are added to the county_name.shp file, the file is finished; but it still needs to be converted to a .geojson, so that is accomplished right before the master loops moves on to the next unique FIPS code

In [1]:
# Setup
# this line is to be able to use the call() to convert the .shp to .geojson later on at the end.
from subprocess import call
import os
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\ArcToolbox\\Scripts')
import arcpy
from arcpy import env
env.overwriteOutput = True
os.environ["GDAL_DATA"] = "C:\\OSGeo4W\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W\\apps\\msys\\bin;C:\\OSGeo4W\\apps\\Python27\\Scripts"

def makeallthethings(mypath, myfile, mytable):
    
    env.workspace = mypath
    
    cursor1 = arcpy.da.SearchCursor(myfile, ["COUNTYFP10"])
    
    countyfips = []
    
# First Loop:
# Getting a list of all the unique county FIPS codes together       
    for row1 in cursor1:
        if row1[0] not in countyfips:
            countyfips.append(row1[0])
    del row1
    del cursor1
    
    # ["001", "003", etc] is what countyfips looks like by this line
    # for debugging, the following was added to make sure the countyfips list was written correctly
    # print countyfips
    
# Second Loop:
# Getting the many .geojson files together
    
    # for every unique FIPS number...
    for fipsnum in countyfips:
        cursor2 = arcpy.da.SearchCursor(mytable, ["CountyName", "FIPSCounty"])
        county_name = ""
        # look through the WashingtonFIPS database file...
        for row2 in cursor2:
            # And find the matching County Name (there should only be one match for this file).
            if fipsnum == row2[1]:
                county_name = row2[0]
        del row2
        del cursor2
        
        # This sets up (for example on the FIPS code 001), Adams.shp, Adams.geojson, C:\\desktop\\temp\\Adams.shp, and 
        # C:\\desktop\\temp\\Adams.geojson as variables to be used later in the cursor and CreateFeatureclass_management calls
        county_file_name = county_name + ".shp"
        county_geo = county_name + ".geojson"
        county_geo_path = mypath + "\\" + county_geo
        county_file_path = mypath + "\\" + county_file_name
        
        # for debugging, the following was added to check to see if the file name was made correctly
        # print county_file_name
        
        # for FIPS code 001 by this line the file Adams.shp will be created, using the "saep_bg10" file
        # as a means to copy the schema and the spatial references
        arcpy.CreateFeatureclass_management(mypath, county_file_name, "POLYGON", myfile, "", "", myfile)
    
        # cursors don't read the geometry, so "SHAPE@" is needed in order to copy that over as well.
        cursor1 = arcpy.da.SearchCursor(myfile, ["SHAPE@", "COUNTYFP10", "POP2013"])
        cursor2 = arcpy.da.InsertCursor(county_file_name, ["SHAPE@", "COUNTYFP10", "POP2013"])
        
        for row1 in cursor1:
            # for debugging, the following was added to check if row1[3] and fipsnum are being read properly in the if test
            # print row1[3]
            # print fipsnum
            if row1[1] == fipsnum:
                cursor2.insertRow(row1)
        del row1
        del cursor1
        del cursor2
        
        # Where it says "'-t_srs','WGS84'", that's what they want the output projection for the .geojson to be, 
        # 'WGS84 being the code name for the certain projection,
        # Where it says "'-s_srs','EPSG:2927'", that's what the "saep_bg10.shp" projection is, which from the original
        # code posted on the assignment page in canvas is actually a different code, so we had to go to a link provided
        # on that same page called "this handy link". On there we had to find a .prj file for one of our county .shp files,
        # I just happened to use Asotin.prj, and that site took that file and figured out the proper code to plug into this
        # call command, which happens to be 'EPSG:2927'. 
        # Using the imported ogr2ogr lines at the top of the code, this call() function converts the named .shp into a named
        # .geojson file, but doesn't actually overwrite the .shp file. In the temp folder there is both a .shp and a 
        # .geojson file. 
        call(['C:\\OSGeo4W\\bin\\ogr2ogr',
      '-f','GeoJSON','-t_srs','WGS84',
      '-s_srs','EPSG:2927',
      county_geo_path,
      county_file_path])
        
        print county_geo + " was created!"
        
# ------------------------------------------------------------------------------------------------------------ #                 



makeallthethings("C:\\Users\\osterj\\Desktop\\temp", "saep_bg10.shp", "WashingtonFIPS.dbf")     

Adams.geojson was created!
Asotin.geojson was created!
Benton.geojson was created!
Chelan.geojson was created!
Clallam.geojson was created!
Clark.geojson was created!
Columbia.geojson was created!
Cowlitz.geojson was created!
Douglas.geojson was created!
Ferry.geojson was created!
Franklin.geojson was created!
Garfield.geojson was created!
Grant.geojson was created!
Grays Harbor.geojson was created!
Island.geojson was created!
Jefferson.geojson was created!
King.geojson was created!
Kitsap.geojson was created!
Kittitas.geojson was created!
Klickitat.geojson was created!
Lewis.geojson was created!
Lincoln.geojson was created!
Mason.geojson was created!
Okanogan.geojson was created!
Pacific.geojson was created!
Pend Oreille.geojson was created!
Pierce.geojson was created!
San Juan.geojson was created!
Skagit.geojson was created!
Skamania.geojson was created!
Snohomish.geojson was created!
Spokane.geojson was created!
Stevens.geojson was created!
Thurston.geojson was created!
Wahkiakum.ge

# Part 2: 
## Print out a ranked list, in descending order, 
## of the ten largest total populations in 2013 of counties in Washington,
## according to this dataset.

This second part of the deliverables I say took me at most, an hour to complete (somehow)
I only used the aid of google too!

At first I was going to make 3 lists, one of the FIPS codes, one of the county names, and a third of the total populations -- to then join in a master dictionary which I would print out. 

I couldn't figure out how to join them together, and a dictionary is only a key with a value, not a 3rd item (that's a tuple?), so I decided to in 1 loop: find a given FIPS code's county name, (in a nested loop) calculate the total population for that county, and add those two values together into a dictionary.

Then I would sort the dictionary in descending order so I could loop through it in order to print out the first 10 entries (the top 10 most populous).

I think it is possible to nest this function (as long as the "for operator import itemgetter" line was added to the top and not nested inside) inside the "makeallthethings(mypath, myfile, mytable)" and have it called at the end of the function so that when "makeallthethings(mypath, myfile, mytable)" is called the code will make all the .geojson files as well as spit out the top 10 list all in 1 go, but eeeeh.

In [2]:
# Setup
import os
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\ArcToolbox\\Scripts')
import arcpy
from arcpy import env
env.overwriteOutput = True
# In order to sort the dictionary we need this line
from operator import itemgetter

def poptotalslist(mypath, myfile, mytable):
    env.workspace = mypath
    
    cursor1 = arcpy.da.SearchCursor(myfile, ["COUNTYFP10"])
    
    countyfips = []
    
    # gets the county fips together in a list
    for row1 in cursor1:
        if row1[0] not in countyfips:
            countyfips.append(row1[0])
    del row1
    del cursor1
    
    # Now we have a list of all the unique county FIPS codes that looks like
    # ["001", "003", "005", etc, etc] by this line
    
    # for debugging purposes the following was added to check to see if the FIPS list was made properly
    # print countyfips
    # print ""
    # print ""
    
    county_namePop = {}
    
    # Makes a dictionary that *should* look like ["Adams": 19200.0, "Asotin": 21799.9999, etc:etc, etc:etc]
    
    # for each unique FIPS code...
    for fipsnum in countyfips:
        cursor2 = arcpy.da.SearchCursor(mytable, ["CountyName", "FIPSCounty"])
        county_name = ""
        
        # gets the unique county name
        for row2 in cursor2:
            if fipsnum == row2[1]:
                # Now we have a saved variable that for ex would say "Adams" because it will only match once with the 
                # whole WashingtonFIPS.dbf file.
                county_name = row2[0]
        del row2
        del cursor2
        
        county_pop = 0
        
        cursor3 = arcpy.da.SearchCursor(myfile, ["COUNTYFP10", "POP2013"])
        
        # totals up all the populations for each county
        for row3 in cursor3:  
            if fipsnum == row3[0]:
                # After this loop we'll have the total of the county we're currently on assigned to the variable county_pop
                county_pop += row3[1]        
        del row3
        del cursor3
        
        # Appends to the dictionary
        county_namePop.update({county_name:county_pop})
    
    # Now we have a dictionary that goes {'San Juan': 16000, 'King': 1981899.999, etc:etc, etc:etc} at this line
    
    # for debugging purposes the following was added to check to see if the dictionary was made properly
    # print county_namePop
    # print ""
    # print ""
    
    # Orders the list in ascending order and overwrites the previous order
    # sorted defaults to ascending order so the "reverse = True" parameter switches the order to descending.
    county_namePop = sorted(county_namePop.items(), key = itemgetter(1), reverse = True)
    
    # for debugging purposes the following was added to check to see if the dictionary was sorted properly
    # print county_namePop
    # print ""
    # print ""
    
    # Prints out an ordered list in descending order,
    # X Population in Y County
    print "From most populous to least, here is a list of the 10 most"
    print "populous counties in Washington State:"
    print "----------------------------------------------------------------"
    print ""
    rank = 1
    # county_namePop[:10] forces the loop to stop at the 10th entry (index 0 through index 9). 
    # county is the key, with pop being the value
    for county, pop in county_namePop[:10]:
        print "Rank " + str(rank) + ": " + county + " County with a total population of ~" + str(pop)
        rank += 1  
        
   
    
    
# ------------------------------------------------------------------------------------------------------------------------ #    



poptotalslist("C:\\Users\\osterj\\Desktop\\temp", "saep_bg10.shp", "WashingtonFIPS.dbf")

From most populous to least, here is a list of the 10 most
populous counties in Washington State:
----------------------------------------------------------------

Rank 1: King County with a total population of ~1981900.0
Rank 2: Pierce County with a total population of ~814500.0
Rank 3: Snohomish County with a total population of ~730500.0
Rank 4: Spokane County with a total population of ~480000.0
Rank 5: Clark County with a total population of ~435500.0
Rank 6: Thurston County with a total population of ~260100.0
Rank 7: Kitsap County with a total population of ~254000.0
Rank 8: Yakima County with a total population of ~247250.0
Rank 9: Whatcom County with a total population of ~205800.0
Rank 10: Benton County with a total population of ~183400.0
